<a href="https://colab.research.google.com/github/RenatoCes/Trabalho_Final_IA/blob/Felipe/Reconhecimento_Facial_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install opencv-python-headless face_recognition matplotlib

In [ ]:

import cv2
import face_recognition
import numpy as np
import os
import base64
from IPython.display import display, Javascript
from google.colab.output import eval_js
import matplotlib.pyplot as plt

# Carregar o classificador Haar Cascade
alg = "/content/haarcascade_frontalface_default.xml"
haar_cascade = cv2.CascadeClassifier(alg)

# Função para codificar uma face usando face_recognition
def encode_face(image):
    if image is None:
        return None
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    face_encodings = face_recognition.face_encodings(rgb_image)
    if face_encodings:
        return face_encodings[0]
    return None

# Carregar faces armazenadas
stored_faces_dir = "stored_faces"
known_faces = {}
if not os.path.exists(stored_faces_dir):
    os.makedirs(stored_faces_dir)

for filename in os.listdir(stored_faces_dir):
    if filename.endswith(".jpg"):
        name = os.path.splitext(filename)[0]
        stored_face = face_recognition.load_image_file(os.path.join(stored_faces_dir, filename))
        encoding = encode_face(stored_face)
        if encoding is not None:
            known_faces[name] = encoding

# Função para capturar uma imagem da webcam usando JavaScript
def capture_image():
    js = Javascript('''
        async function captureImage() {
            const video = document.createElement('video');
            const stream = await navigator.mediaDevices.getUserMedia({video: true});
            video.srcObject = stream;
            await new Promise((resolve) => {
                video.onloadedmetadata = () => {
                    resolve(video);
                };
            });
            document.body.appendChild(video);
            video.play();

            // Create a canvas to capture the frame
            const canvas = document.createElement('canvas');
            canvas.width = video.videoWidth;
            canvas.height = video.videoHeight;
            const context = canvas.getContext('2d');
            context.drawImage(video, 0, 0, canvas.width, canvas.height);
            
            // Stop the video stream
            stream.getTracks().forEach(track => track.stop());
            video.remove();

            return canvas.toDataURL('image/jpeg', 1.0);
        }
        captureImage();
    ''')
    display(js)
    img_str = eval_js('captureImage()')
    img_data = img_str.split(',')[1]
    nparr = np.frombuffer(base64.b64decode(img_data), np.uint8)
    frame = cv2.imdecode(nparr, cv2.IMREAD_COLOR)
    return frame

# Função para exibir a imagem capturada
def display_image(image):
    rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    plt.imshow(rgb_image)
    plt.axis('off')
    plt.show()

# Função para verificar se a face está registrada
def check_face_registration(frame, input_name=None):
    if frame is None:
        print("Erro: Imagem não capturada corretamente.")
        return None

    display_image(frame)  # Mostrar a imagem capturada

    new_face_encoding = encode_face(frame)
    if new_face_encoding is not None:
        matches = face_recognition.compare_faces(list(known_faces.values()), new_face_encoding)
        if any(matches):
            match_index = matches.index(True)
            match_name = list(known_faces.keys())[match_index]
            print(f"Face correspondente encontrada: {match_name}")
            if input_name is not None:
                if input_name == match_name:
                    print(f"Nome verificado com sucesso: {input_name} é {match_name}")
                    return match_name
                else:
                    print(f"Erro na verificação: {input_name} não corresponde a {match_name}")
                    return None
        else:
            print("Nenhuma correspondência encontrada para esta face.")
            return None
    else:
        print("Não foi possível codificar a face.")
        return None

# Função para verificar uma imagem externa
def verify_external_image(image_path, real_name):
    if not os.path.exists(image_path):
        print("Erro: Caminho da imagem não existe.")
        return None
    image = face_recognition.load_image_file(image_path)
    return check_face_registration(image, real_name)

# Função para calcular a acurácia
def calcular_acuracia(predicoes, etiquetas_reais):
    corretas = sum(1 for p, e in zip(predicoes, etiquetas_reais) if p == e)
    acuracia = corretas / len(etiquetas_reais) if etiquetas_reais else 0
    return acuracia

# Função principal
def main():
    answer = input("Verificar se o seu rosto está registrado? sim, não ou teste? ").strip().lower()
    if answer == "não":
        print("Programa encerrado.")
    elif answer == "sim":
        name = input("Digite seu nome: ").strip()
        print("Ligando a câmera...")
        frame = capture_image()
        print("Verificando face...")
        check_face_registration(frame, name)
    elif answer == "teste":
        # Solicita o nome da pasta ao usuário
        nome_pasta = "faces"

        # Verifica se a pasta existe
        if not os.path.isdir(nome_pasta):
            print(f"A pasta {nome_pasta} não existe.")
        else:
            # Obtém a lista de arquivos na pasta
            arquivos = os.listdir(nome_pasta)

            # Filtra apenas os arquivos com extensão .jpg
            fotos = [arquivo for arquivo in arquivos if arquivo.endswith('.jpg')]

            if not fotos:
                print("Não há fotos com extensão .jpg na pasta.")
            else:
                etiquetas_reais = []
                predicoes = []

                # Teste para cada foto
                for foto in fotos:
                    caminho_completo = os.path.join(nome_pasta, foto)
                    image_path = caminho_completo.strip()
                    real_name = os.path.splitext(foto)[0]
                    etiquetas_reais.append(real_name)
                    predicao = verify_external_image(image_path, real_name)
                    predicoes.append(predicao)

                # Calcula e exibe a acurácia
                acuracia = calcular_acuracia(predicoes, etiquetas_reais)
                print(f"Acurácia: {acuracia * 100:.2f}%")
    else:
        print("Resposta inválida. Programa encerrado.")

# Executar o programa principal
main()
